In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import logging

# logger = logging.basicConfig(level=logging.INFO)

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

## install dependencies

In [ ]:
# !mamba install -c conda-forge -y make proj-data sentinelsat xmlschema

In [ ]:
# !pip install elevation sarsen xarray-sentinel

In [ ]:
import os

import numpy as np
import rioxarray  # enable the `.rio` accessor
import xarray as xr

from sarsen import apps, scene

## processing definition

In [ ]:
os.makedirs("data", exist_ok=True)
dem_urlpath = "data/South-of-Redmond-10m.tif"
dem_10m_UTM_urlpath = dem_urlpath.strip(".tif") + "_UTM.tif"

product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"
measurement_group = "IW/VV"

## download DEM

In [ ]:
import adlfs
import planetary_computer
import pystac_client
import stackstac

In [ ]:
seattle = [-121.95, 47.04]
areas_of_interest = {"type": "Point", "coordinates": seattle}

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
threedep = catalog.get_child(id="3dep-seamless")

longitude, latitude = seattle
buffer = 0.2
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())
items

In [ ]:
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

In [ ]:
dem_raster = dem_raster_all.compute()
if "time" in dem_raster.dims:
    dem_raster = dem_raster.mean("time")
dem_raster.rio.set_crs(dem_raster_all.rio.crs)
dem_raster.rio.to_raster(dem_urlpath)

### RESAMPLING UTM DEM

#### 10 meters

In [ ]:
dem_raster = scene.open_dem_raster(dem_urlpath)
_ = dem_raster.plot()

In [ ]:
dem_UTM_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))

resolution = (10, 10)

t_srs = dem_raster.rio.estimate_utm_crs()
dem_raster_10m_UTM = dem_raster.rio.reproject(t_srs, resolution=resolution)

dem_raster_10m_UTM = dem_raster_10m_UTM.sel(**dem_UTM_corners)

dem_raster_10m_UTM.rio.to_raster(dem_10m_UTM_urlpath)
dem_raster_10m_UTM

In [ ]:
dem_raster_10m_UTM.plot()

## download data

In [ ]:
import os


def mirror_folder(fs, bucket, folder, exclude="vh"):
    for path, folders, files in fs.walk(f"{bucket}/{folder}"):
        os.makedirs(path[len(bucket) + 1 :], exist_ok=True)
        for f in files:
            if exclude in f:
                continue
            file_path = os.path.join(path, f)
            lfile_path = file_path[len(bucket) + 1 :]
            if not os.path.isfile(lfile_path):
                print(file_path)
                fs.download(file_path, lfile_path + "~")
                os.rename(lfile_path + "~", lfile_path)

In [ ]:
grd_account_name = "sentinel1euwest"
grd_bucket = "s1-grd"
grd_token = planetary_computer.sas.get_token(grd_account_name, grd_bucket).token

grd_product_folder = f"{grd_bucket}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(grd_product_folder)

In [ ]:
mirror_folder(grd_fs, grd_bucket, product_folder)

In [ ]:
ds = xr.open_dataset(
    product_folder,
    engine="sentinel-1",
    group=measurement_group,
    # storage_options=storage_options,
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
)
ds

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_bucket = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(rtc_account_name, rtc_bucket).token

rtc_product_folder = f"{rtc_bucket}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
mirror_folder(rtc_fs, rtc_bucket, product_folder)

## process

#### GTC

In [ ]:
%%time

gtc_path = os.path.basename(product_folder) + ".10m.GTC.tif"

apps.terrain_correction(
    product_folder,
    measurement_group,
    dem_10m_UTM_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
    output_urlpath=gtc_path,
)

!gdaladdo -r average {gtc_path}

gtc_path

In [ ]:
gtc = xr.open_dataarray(gtc_path).drop("band")
gtc.plot(vmax=0.4);

#### RTC

In [ ]:
%%time

rtc_path = os.path.basename(product_folder) + ".10m.RTC.tif"

apps.terrain_correction(
    product_folder,
    measurement_group,
    dem_10m_UTM_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    correct_radiometry="gamma_bilinear",
    output_urlpath=rtc_path,
    grouping_area_factor=(3, 3),
)

!gdaladdo -r average {rtc_path}

rtc_path

In [ ]:
rtc = xr.open_dataarray(rtc_path, cache=False).drop("band")
rtc.plot(vmax=0.4);

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()

In [ ]:
rtc_pc = xr.open_dataarray(
    product_folder + "/measurement/iw-vv.rtc.tiff", cache=False
).drop("band")
rtc_pc = rtc_pc.sel(dem_UTM_corners)
rtc_pc

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()